# This notebook contains code for the blog "Changing the Frequency (Precision) of Time Data in Pandas: Using Pandas' resample() method to adjust time features"

## import necessary packages

In [1]:
import pandas as pd

## Load data

In [21]:
df = pd.read_csv('edmonds.csv')

In [22]:
df = df.sort_values('time')
df = df.reset_index(drop=True)

In [23]:
df.head(10)

,tweet_text,time,wait_time
0,ed/ki- edmonds wait time - 60 minutes,2016-12-24 20:50 +0000,1.0
1,ed/king- edmonds wait time - 60 minutes,2016-12-24 20:55 +0000,1.0
2,ed/king- edmonds - no extended wait,2016-12-24 23:40 +0000,0.0
3,edm/ki- edmonds terminal wait time: 1 hour,2016-12-25 20:15 +0000,1.0
4,update - no extended wait departing edmonds,2016-12-26 00:30 +0000,0.0
5,edmonds terminal wait time - 1 hour,2016-12-26 20:30 +0000,1.0
6,update - no extended wait departing edmonds,2016-12-27 02:05 +0000,0.0
7,edmonds terminal wait time - one hour,2016-12-30 00:35 +0000,1.0
8,update - no extended wait departing edmonds,2016-12-30 03:40 +0000,0.0
9,ed/ki - edmonds terminal - one hour wait,2016-12-30 23:25 +0000,1.0


## The Goal

In [24]:
df_goal = pd.DataFrame(pd.date_range(start='2016-12-24 20:00 +0000', end='2016-12-25 15:00 +0000', freq='H'))
df_goal['wait_time'] = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
df_goal.columns = ['time', 'wait_time']

In [25]:
df_goal

,time,wait_time
0,2016-12-24 20:00:00+00:00,1
1,2016-12-24 21:00:00+00:00,1
2,2016-12-24 22:00:00+00:00,1
3,2016-12-24 23:00:00+00:00,1
4,2016-12-25 00:00:00+00:00,0
5,2016-12-25 01:00:00+00:00,0
6,2016-12-25 02:00:00+00:00,0
7,2016-12-25 03:00:00+00:00,0
8,2016-12-25 04:00:00+00:00,0
9,2016-12-25 05:00:00+00:00,0


## Convert to pacific time

In [26]:
# convert to datetime
df['time'] = pd.to_datetime(df['time'], utc=True)

# set time as index
df = df.set_index('time')

# convert to US/Pacific time zone
df = df.tz_convert('US/Pacific')

## Add start of day, no wait - assuming waits reset over night

In [27]:
# create a dataframe with 4am of each day
sod = pd.date_range('2016-12-24 04:00:00', '2019-12-31 4:00:00', freq='D')
sod_df = pd.DataFrame(sod)
sod_df.columns = ['time']
sod_df = sod_df.set_index('time')

# set the wait time to 0
sod_df['wait_time'] = 0

# localize the start of day dataframe to pacific time
sod_df.index = sod_df.index.tz_localize('US/Pacific')

# append the start of day dataframe to the original and re-sort
df = df.append(sod_df)
df = df.sort_values('time')

## Resample datetime

In [29]:
# resample times to hour intervals, adding the "missing" hours, 
# and filling the wait time forward
df = df.resample('1H').ffill()

In [30]:
df.head(10)

,tweet_text,wait_time
time,,
2016-12-24 04:00:00-08:00,NaN,0.0
2016-12-24 05:00:00-08:00,NaN,0.0
2016-12-24 06:00:00-08:00,NaN,0.0
2016-12-24 07:00:00-08:00,NaN,0.0
2016-12-24 08:00:00-08:00,NaN,0.0
2016-12-24 09:00:00-08:00,NaN,0.0
2016-12-24 10:00:00-08:00,NaN,0.0
2016-12-24 11:00:00-08:00,NaN,0.0
2016-12-24 12:00:00-08:00,NaN,0.0


## Remove non-running hours

In [31]:
# remove non-sailing times (1-4am for Edmonds, 1-3 am for Kingston)
df = df.between_time('5:00', '0:55')

In [32]:
df.head(10)

,tweet_text,wait_time
time,,
2016-12-24 05:00:00-08:00,NaN,0.0
2016-12-24 06:00:00-08:00,NaN,0.0
2016-12-24 07:00:00-08:00,NaN,0.0
2016-12-24 08:00:00-08:00,NaN,0.0
2016-12-24 09:00:00-08:00,NaN,0.0
2016-12-24 10:00:00-08:00,NaN,0.0
2016-12-24 11:00:00-08:00,NaN,0.0
2016-12-24 12:00:00-08:00,NaN,0.0
2016-12-24 13:00:00-08:00,ed/king- edmonds wait time - 60 minutes,1.0
